In [12]:
import pandas
import requests
from bs4 import BeautifulSoup

In [14]:
x = requests.get("https://en.wiktionary.org/wiki/Category:Vietnamese_lemmas")
soup = BeautifulSoup(x.text)

In [53]:
soup.find_all('div', class_='mw-content-ltr')[-1].find_all('li')

[<li><a href="/wiki/9x" title="9x">9x</a></li>,
 <li><a href="/wiki/A" title="A">A</a></li>,
 <li><a href="/wiki/a" title="a">a</a></li>,
 <li><a href="/wiki/a_c%C3%B2ng" title="a còng">a còng</a></li>,
 <li><a href="/wiki/A_Di_%C4%90%C3%A0" title="A Di Đà">A Di Đà</a></li>,
 <li><a href="/wiki/A_Di_%C4%90%C3%A0_Ph%E1%BA%ADt" title="A Di Đà Phật">A Di Đà Phật</a></li>,
 <li><a href="/wiki/a_dua" title="a dua">a dua</a></li>,
 <li><a href="/wiki/a_giao" title="a giao">a giao</a></li>,
 <li><a href="/wiki/a_ha" title="a ha">a ha</a></li>,
 <li><a href="/wiki/a_ho%C3%A0n" title="a hoàn">a hoàn</a></li>,
 <li><a href="/wiki/A_La_H%C3%A1n" title="A La Hán">A La Hán</a></li>,
 <li><a href="/wiki/A_L%E1%BB%8Bch_S%C6%A1n" title="A Lịch Sơn">A Lịch Sơn</a></li>,
 <li><a href="/wiki/A_L%E1%BB%8Bch_S%C6%A1n_%C4%90%E1%BA%AFc_L%E1%BB%99" title="A Lịch Sơn Đắc Lộ">A Lịch Sơn Đắc Lộ</a></li>,
 <li><a href="/wiki/a_l%C3%B4" title="a lô">a lô</a></li>,
 <li><a href="/wiki/a_m%C3%B3c" title="a móc">a mó